In [1]:
import torch
import torch.nn as nn
import numpy as np
import sys
import os
import random
import matplotlib.pyplot as plt

In [2]:
# check if GPU is available and set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


### Model + Helper Functions

In [3]:
# Please put the src directory in the path
src_path = os.path.abspath(os.path.join(os.getcwd(), 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

from utils import MIMONetDataset, ChannelScaler
from mimonet import MIMONet

In [4]:
# set working directory
working_dir = "/projects/bcnx/kazumak2/MIMONet/HeatExchanger"
data_dir = os.path.join(working_dir, "data")

### Load datasets

In [5]:
# trunk dataset
trunk_input = np.load(os.path.join(data_dir, "share/trunk.npz"))['trunk']

# min-max scaling [-1, 1]
trunk_input[:, 0] = 2 * (trunk_input[:, 0] - np.min(trunk_input[:, 0])) / (np.max(trunk_input[:, 0]) - np.min(trunk_input[:, 0])) - 1
trunk_input[:, 1] = 2 * (trunk_input[:, 1] - np.min(trunk_input[:, 1])) / (np.max(trunk_input[:, 1]) - np.min(trunk_input[:, 1])) - 1

In [6]:
# branch input dataset
branch = np.load(os.path.join(data_dir, "branch.npz"))

branch1 = branch['branch1']
branch2 = branch['branch2']

print("Branch1 shape:", branch1.shape)
print("Branch2 shape:", branch2.shape)

# split the dataset into training and testing sets
train_size = int(0.8 * len(branch1))
test_size = len(branch1) - train_size
train_branch1, test_branch1 = branch1[:train_size], branch1[train_size:]
train_branch2, test_branch2 = branch2[:train_size], branch2[train_size:]

Branch1 shape: (1546, 2)
Branch2 shape: (1546, 100)


In [7]:
# create a dictionary for the output channel names
# 0: turb-kinetic-energy
# 1: pressure
# 2: temperature
# 3: z-velocity
# 4: y-velocity
# 5: x-velocity
# 6: velocity-magnitude

dict_channel = {
    0: 'turb-kinetic-energy',
    1: 'pressure',
    2: 'temperature',
    3: 'z-velocity',
    4: 'y-velocity',
    5: 'x-velocity',
    6: 'velocity-magnitude'
}

# select the output channel
target_channel = [1, 3, 4, 5, 6]

# print the selected output channel names
# target_label is used to store the names of the selected output channels for further processing (e.g., plotting)
print("Selected output channels:")
target_label = []
for channel in target_channel:
    print(dict_channel[channel])
    target_label.append(dict_channel[channel])    

Selected output channels:
pressure
z-velocity
y-velocity
x-velocity
velocity-magnitude


In [8]:
# target dataset
target = np.load(os.path.join(data_dir, "target.npy"))

# extract the output channels
target = target[:, :, target_channel ]  # select the first 7 channels
print("Target dataset shape before split:", target.shape)


# split the target dataset into training and testing sets
train_target = target[:train_size]
test_target = target[train_size:]

print("Train target shape:", train_target.shape)
print("Test target shape:", test_target.shape)

Target dataset shape before split: (1546, 3977, 5)
Train target shape: (1236, 3977, 5)
Test target shape: (310, 3977, 5)


In [9]:
# get the mean and standard deviation of each channel
mean_branch1 = np.mean(train_branch1, axis=0)
std_branch1 = np.std(train_branch1, axis=0)

print("Mean of branch1:", mean_branch1)
print("Std of branch1:", std_branch1)

# (# train samples, 100)
mean_branch2 = np.mean(train_branch2)
std_branch2 = np.std(train_branch2)

print("Mean of branch2:", mean_branch2)
print("Std of branch2:", std_branch2)

Mean of branch1: [  4.51454429 292.42944177]
Std of branch1: [ 0.2615285  17.03323994]
Mean of branch2: 12587.66968713018
Std of branch2: 6302.709013835411


In [10]:
# normalize the branch data using the mean and std
train_branch_1 = (train_branch1 - mean_branch1) / std_branch1
test_branch_1 = (test_branch1 - mean_branch1) / std_branch1
train_branch_2 = (train_branch2 - mean_branch2) / std_branch2
test_branch_2 = (test_branch2 - mean_branch2) / std_branch2

# print the shapes of the normalized data
print("Shape of normalized train_branch1:", train_branch_1.shape)
print("Shape of normalized test_branch1:", test_branch_1.shape)
print("Shape of normalized train_branch2:", train_branch_2.shape)
print("Shape of normalized test_branch2:", test_branch_2.shape)

Shape of normalized train_branch1: (1236, 2)
Shape of normalized test_branch1: (310, 2)
Shape of normalized train_branch2: (1236, 100)
Shape of normalized test_branch2: (310, 100)


### Scaling the target data

In [11]:
# scaling the target data
'''  
note: reverse the scaling for the target data
train_target = scaler.inverse_transform(train_target_scaled)
test_target = scaler.inverse_transform(test_target_scaled)
'''
scaler = ChannelScaler(method='minmax', feature_range=(-1, 1))
scaler.fit(train_target)
train_target_scaled = scaler.transform(train_target)
test_target_scaled = scaler.transform(test_target)

print("Shape of scaled train_target:", train_target_scaled.shape)
print("Shape of scaled test_target:", test_target_scaled.shape)

Shape of scaled train_target: (1236, 3977, 5)
Shape of scaled test_target: (310, 3977, 5)


In [12]:
# test dataset and dataloader
test_dataset = MIMONetDataset(
    [test_branch_1, test_branch_2],  # branch_data_list
    trunk_input,                     # trunk_data
    test_target_scaled               # target_data
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=1,  # set to 1 for testing
    shuffle=False,
    num_workers=0
)

## Model Instance & Load Parameters

In [13]:
# Architecture parameters
dim = 256
branch_input_dim1 = 2
branch_input_dim2 = 100
trunk_input_dim = 2

# Define the model arguments for orig_MIMONet
model_args = {
    'branch_arch_list': [
        [branch_input_dim1, 512, 512, 512, dim],
        [branch_input_dim2, 512, 512, 512, dim]
    ],
    'trunk_arch': [trunk_input_dim, 256, 256, 256, dim],
    'num_outputs': target.shape[-1] -1,  # number of output channels
    'activation_fn': nn.ReLU,
    'merge_type': 'mul',
}

### Load

In [14]:
model = MIMONet(**model_args).to(device)
# Load the trained model
model_path = os.path.join(working_dir, "checkpoints/custom_best_model_lambda_1E-4.pt")

if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location=device))
    print("Model loaded successfully from", model_path)
else:
    print("Model file not found at", model_path)
    sys.exit(1)
    
# Evaluate the model on the test set
model.eval()

# print the number of parameters in the model
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Number of trainable parameters in the model: {num_params}")

Model loaded successfully from /projects/bcnx/kazumak2/MIMONet/HeatExchanger/checkpoints/custom_best_model_lambda_1E-4.pt
Number of trainable parameters in the model: 1762052


## How to predict?

In [15]:
# feed the test loader to the model (and save predictions and targets)
predictions = []
targets = []
with torch.no_grad():
    for batch in test_loader:
        branch_data, trunk_data, target_data = batch
        branch_data = [b.to(device) for b in branch_data]
        trunk_data = trunk_data.to(device)
        target_data = target_data.to(device)

        output = model(branch_data, trunk_data)
        predictions.append(output.cpu().numpy())
        targets.append(target_data.cpu().numpy())
        
# Convert predictions and targets to numpy arrays
predictions = np.concatenate(predictions, axis=0)
targets = np.concatenate(targets, axis=0)

# Reverse the scaling for the target data
predictions = scaler.inverse_transform(predictions)
targets = scaler.inverse_transform(targets)

(Evaluate)

In [16]:
# Compute L2 norm over grid points for each sample and channel
l2_pred = np.linalg.norm(predictions, axis=1)  # shape: (samples, channels)
l2_gt = np.linalg.norm(targets[..., :4], axis=1)  # shape: (samples, channels)

print("L2 norm of predictions shape:", l2_pred.shape)
print("L2 norm of targets shape:", l2_gt.shape)


# Compute L2 error over grid points for each sample and channel
l2_err = np.linalg.norm(predictions - targets[..., :4], axis=1)  # shape: (samples, channels)

# Compute relative error (avoid division by zero)
rel_err = l2_err / (l2_gt + 1e-8)  # shape: (samples, channels)

# Mean over samples for each channel
mean_rel_err_per_channel = np.mean(rel_err, axis=0)  # shape: (channels,)

print("Mean relative L2 error per channel:", mean_rel_err_per_channel * 100, "%")

# standard deviation of relative error per channel
std_rel_err_per_channel = np.std(rel_err, axis=0)  # shape: (channels,)

print("Standard deviation of relative L2 error per channel:", std_rel_err_per_channel)

L2 norm of predictions shape: (310, 4)
L2 norm of targets shape: (310, 4)
Mean relative L2 error per channel: [0.79963773 1.4521601  1.015162   0.5177918 ] %
Standard deviation of relative L2 error per channel: [0.00014584 0.0002911  0.0002742  0.00038389]
